In [1]:
Preparing Data
==============

SyntaxError: invalid syntax (<ipython-input-1-c6701043bd06>, line 1)

In [2]:
import json
# import cPickle as pickle
import pickle
import numpy as np
import pandas as pd
from pandas import factorize

In [3]:
!ls bossa/*json

bossa/control_tasks.json       bossa/tasks_export.json
bossa/control_tasks_runs.json  bossa/tasks_runs_export.json


BOSSA Results
-------------

Processing `results_bossa.json` to get a *dictionary* with keys the task ids, and values in as the average value of the scores. To do that, we first convert scores from categorical (`neg`, `neu`, `pos`) to a numeric scale.

In [4]:
bossa_results = pd.read_json("bossa/tasks_runs_export.json")
bossa_results.rename(columns={"created": "start_time", "id": "result_id", "info": "score"}, inplace=True)
bossa_results[['start_time']]= bossa_results[['start_time']].apply(pd.to_datetime, dayfirst=True)
bossa_results[['finish_time']]= bossa_results[['finish_time']].apply(pd.to_datetime, dayfirst=True)
bossa_results['score'] = pd.Categorical(bossa_results['score'], categories=['vneg', 'neg', 'neu', 'pos', 'vpos'])
bossa_results['score'].cat.rename_categories([-2, -1, 0, 1, 2], inplace=True)
# Normalize everything to -1, 0, 1
# bossa_results['score'] = bossa_results['score'].astype(float).apply(lambda x: -1 if x < 0 else 1 if x > 0 else 0)
bossa_results["seconds"] = (bossa_results["finish_time"] - bossa_results["start_time"]).astype('timedelta64[us]') / 1e6
bossa_results = bossa_results[["result_id", "seconds", "task_id", "score"]]
bossa_results.ix[[50]]

,result_id,seconds,task_id,score
50,11203,0.000025,52775,1


The information about the sentence comes in a dictionary inside the cells of the serie `info`, so we expand it.

In [5]:
bossa_tasks = pd.read_json("bossa/tasks_export.json")
bossa_tasks[['created']]= bossa_tasks[['created']].apply(pd.to_datetime, dayfirst=True)
bossa_tasks.rename(columns={'id': 'task_id'}, inplace=True)
bossa_tasks = bossa_tasks[['task_id', 'info']]
bossa_tasks.ix[[50]]

,task_id,info
50,52851,"{'appears_in_noun_phrases': '0', 'is_company':..."


And finally we merge the `DataFrame` with the scores with the one containing the sentences.

In [6]:
bossa_tasks_scores = pd.merge(bossa_results, bossa_tasks, on='task_id')
bossa_tasks_scores.ix[[50]]

,result_id,seconds,task_id,score,info
50,11195,0.000021,52776,2,"{'appears_in_noun_phrases': '0', 'is_company':..."


Let's now expand the column `info` into as many new columns as keys has the dictionary `info`.

In [7]:
bossa_tasks_scores.ix[50].info.keys()

dict_keys(['appears_in_noun_phrases', 'is_company', 'text', 'media', 'pub_date', 'appears_in_sentence', 'url', 'noun_phrases', 'sentence_id', 'sentence', 'search_words'])

In [8]:
def json_to_series(info):
#     keys, values = zip(*info.iteritems())
    keys, values = zip(*info.items())
    return pd.Series(values, index=keys)

bossa_info = bossa_tasks_scores["info"].apply(json_to_series)
bossa_info.reset_index()
bossa = pd.concat([bossa_tasks_scores, bossa_info], axis=1)
bossa.pop("info")
# bossa['id'] = bossa['id'].astype(float)
bossa.ix[50:53]

,result_id,seconds,task_id,score,appears_in_noun_phrases,is_company,text,media,pub_date,appears_in_sentence,url,noun_phrases,sentence_id,sentence,search_words
50,11195,0.000021,52776,2,0,0,Chinese investors are increasingly opting to b...,nyt,2013-05-17T11:47:51Z,0,http://dealbook.nytimes.com/2013/05/17/a-toeho...,"[chinese investors, overseas companies, politi...",14,Chinese investors are increasingly opting to b...,executive
51,11205,0.000018,52776,-1,0,0,Chinese investors are increasingly opting to b...,nyt,2013-05-17T11:47:51Z,0,http://dealbook.nytimes.com/2013/05/17/a-toeho...,"[chinese investors, overseas companies, politi...",14,Chinese investors are increasingly opting to b...,executive
52,11207,0.000017,52776,1,0,0,Chinese investors are increasingly opting to b...,nyt,2013-05-17T11:47:51Z,0,http://dealbook.nytimes.com/2013/05/17/a-toeho...,"[chinese investors, overseas companies, politi...",14,Chinese investors are increasingly opting to b...,executive
53,11209,0.000017,52776,-2,0,0,Chinese investors are increasingly opting to b...,nyt,2013-05-17T11:47:51Z,0,http://dealbook.nytimes.com/2013/05/17/a-toeho...,"[chinese investors, overseas companies, politi...",14,Chinese investors are increasingly opting to b...,executive


Aggregate
---------

We now aggregate calculating the average per `sentence_id` using a group by. In the process, we lose the source of the data, that's why we first have to save it.

In [9]:
bossa.to_csv("sentiment/scores_ungrouped.csv", encoding="utf8")

Finally, we aggregate and create a new `DataFrame` for the different sentences and their score.

In [10]:
from collections import Counter

def majority(series): #receives a Pandas Series
    c = Counter(map(lambda x: 1 if x > 0 else -1 if x < 0 else 0, series))
    commons = c.most_common()
    if len(commons) > 1 and commons[0][1] == commons[1][1]:
        value = -9
    else:
        value = commons[0][0]
    return value
#     return Counter(map(lambda x: 1 if x > 0 else -1 if x < 0 else 0, series)).most_common(1)[0][0]

# score_calculate = np.average
score_calculate = majority

sentences = bossa.groupby(['sentence'])[['score']].aggregate(score_calculate)
sentences.to_csv("sentiment/scores.csv", encoding="utf8")
print(sentences.count())
sentences[1001:1004]

score    8996
dtype: int64


,score
sentence,
"'We must hope after so much prevarication that this time Google's proposals represent a genuine attempt to address the concerns identified,' said David Wood, the legal counsel for Icomp, an industry group backed by Microsoft and a number of other companies.",0
"'We must push our leaders to step up and commit to action,' said Hugh Evans, the founder and chief executive of the charity.",-1
"'We need them to tell the story of how we are making decisions and putting the organization together,' said George Postolos, the Astros' president and chief executive, who added that the team would not want a broadcaster who was uncomfortable explaining the front office's strategy.",-1


Create the tranining and testing sets (data and labels) from a randomized version of the set of assessed sentences.

In [11]:
sentences.reset_index()['score'].value_counts()

 1    3991
-1    2193
-9    1530
 0    1282
dtype: int64

In [12]:
sentences.reset_index().count()

sentence    8996
score       8996
dtype: int64

In [13]:
# raw_scores = sentences.reset_index()
# scores = raw_scores
# print('Zero:', len(scores[scores.score==0]))
# print('Non-zero:', len(scores[scores.score!=0]))

We could consider 3 classes, but it toruns out that using binary classficication seems to produce better results. Still, try multi-classs classifiers is something worth trying.

In [14]:
raw_scores = sentences.reset_index()
scores = raw_scores
scores = scores[scores.score!=0]  # We ignore the neutral sentences
scores = scores[scores.score!=-9]  # And ambiguous sentences
scores['sentiment'] = scores['score'].apply(lambda s: 'pos' if s > 0 else 'neg')
# scores['sentiment'] = scores['score'].apply(lambda s: 'pos' if s > 0 else 'neg' if s < 0 else 0)

In [15]:
sentences_df = scores[['sentence', 'sentiment']].reset_index(drop=True)
# sentences_df = sentences_df.reindex(np.random.permutation(sentences_df.index))

In [16]:
sentences_df.head()

,sentence,sentiment
0,"General Motors will recall nearly 3,200 manua...",neg
1,""" And Aim's problem is that many of its larges...",neg
2,"""'Don't be evil,' he cried, while being chased...",neg
3,"""A lot of companies seem to prefer it to other...",pos
4,"""An entrepreneur may well be unreasonable beca...",neg


#SCIKIT-LEARN

In [17]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.cross_validation import cross_val_score, StratifiedKFold,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.lda import LDA
from sklearn.linear_model import PassiveAggressiveClassifier, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.utils.extmath import density

In [18]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [19]:
from sklearn.cross_validation import train_test_split

# categories = [
#     'pos',
#     'neg',
# ]
# print("Loading sentences for categories:")
# print(categories)
# data_train, data_test = train_test_split(document_df, train_size=0.9, test_size=0.1, random_state=100)
# print('Data loaded:')
# print('Train set: {} samples'.format(len(data_train)))
# print('Test set: {} samples'.format(len(data_test)))

In [20]:
def size_mb(docs):
    return sum(len(s.encode('utf-8')) for s in docs) / 1e6

# data_train_size_mb = size_mb(data_train['sentence'])
# data_test_size_mb = size_mb(data_test['sentence'])

# print("%d documents - %0.3fMB (training set)" % (
#     len(data_train), data_train_size_mb))
# print("%d documents - %0.3fMB (test set)" % (
#     len(data_test), data_test_size_mb))
# print("%d categories" % len(categories))
# print()

In [21]:
metrics.matthews_corrcoef

<function sklearn.metrics.classification.matthews_corrcoef>

In [39]:
###############################################################################
# Benchmark classifiers
def benchmark(name, clf, X_train, X_test, y_train, y_test):
    print("_" * 80)
    print("Training:", name)
    print(clf)
    
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    matthews_corrcoef = metrics.matthews_corrcoef(y_test, pred)
    print("matthews_corrcoef:   %0.3f" % matthews_corrcoef)

#     precision = metrics.precision_score(y_test, pred)
#     print("precision:   %0.3f" % score)

#     recall = metrics.recall_score(y_test, pred)
#     print("recall:   %0.3f" % score)

    print("classification report:")
    print(metrics.classification_report(y_test, pred))

    print("confusion matrix:")
    print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split("(")[0]
    return score, matthews_corrcoef #, precision, recall

In [23]:
# # # Classifiers
# # clf_list = [
# #     (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge classifier"),
# # #     (Perceptron(n_iter=50), "Perceptron"),
# # #     (PassiveAggressiveClassifier(n_iter=50), "Passive-aggressive"),
# # #     (KNeighborsClassifier(n_neighbors=10), "kNN"),
# # #     (RandomForestClassifier(n_estimators=100), "Random forest")
# # ]

# # Classifiers
# clf_list = [
#     (RidgeClassifier(alpha=.00001, tol=1e-2, solver="lsqr"), "Ridge classifier"),
#     (Perceptron(alpha=.00001, n_iter=50), "Perceptron"),
#     (PassiveAggressiveClassifier(n_iter=50), "Passive-aggressive"),
#     (KNeighborsClassifier(n_neighbors=10), "kNN"),
#     (RandomForestClassifier(n_estimators=100), "Random forest"),
#     (LinearSVC(loss='squared_hinge', penalty='l2', dual=False, tol=1e-3), 'Linear SVC'),
#     (SGDClassifier(alpha=.000001, n_iter=50, penalty='l1'), 'SGDClassifier'),
#     (NearestCentroid(), 'Nearest Centroid'),
#     (MultinomialNB(alpha=.00001), 'Multinomial NB'),
#     (BernoulliNB(alpha=.00001), 'Bernoulli NB'),
#     (LinearSVC(penalty="l1", dual=False, tol=1e-3), 'Linear SVC'),
# ]

In [24]:
# with open('sentiment/myclf.pickle', 'wb') as myclf_file:
#     pickle.dump(pipeline, myclf_file, protocol=2)

In [25]:
# with open('sentiment/myclf.pickle', 'rb') as myclf_file:
#     my_clf = pickle.load(myclf_file)

In [26]:
# # Params
# K = 10
# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 1000, 2000),
#     'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
#     'vect__stop_words': (None, stopwords.words('english')),
#     'tfidf__use_idf': (True, False),
#     'clf__metric': ('euclidean', 'manhattan'),
# #     'clf__shrink_threshold': (None, 0.1),
# }

In [27]:
# X = sentences_df['sentence']
# y = sentences_df['sentiment']

# skf = StratifiedKFold(y, K)
# pipeline = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', NearestCentroid()),
# ])
# grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=skf)
# grid_search.fit(X, y)
# bs = grid_search.best_score_
# be = grid_search.best_estimator_

# print('*'*40)
# print('Best score:', bs)
# print()
# print('Best parameters:')
# print(be.get_params())
# print('*'*40)

In [28]:
class Densifier(object):
    def fit(self, X, y=None):
        pass
    def fit_transform(self, X, y=None):
        return self.transform(X)
    def transform(self, X, y=None):
        return X.toarray()

In [29]:
# # LDA params
# K = 10
# lda_parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 1000, 2000),
#     'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
#     'vect__stop_words': (None, stopwords.words('english')),
#     'tfidf__use_idf': (True, False),
#     'clf__solver': ('svd', 'lsqr', 'eigen'),
#     'clf__shrinkage': (None, 'auto', 0.1, 0.5, 0.9),
#     'clf__tol': (0.001, 0.0001),
# }

In [46]:
# lda_clf_list = [
# #     (LDA(), "LDA"),
#     (LDA(solver='svd', shrinkage=None, tol=1e-4, ), "LDA 1"),
# #     (LDA(solver='svd', shrinkage=None, tol=1e-3, ), "LDA 2"),
# #     (LDA(solver='svd', shrinkage=None, tol=1e-2, ), "LDA 3"),
# #     (LDA(solver='svd', shrinkage=None, tol=1e-1, ), "LDA 4"),
#     (LDA(solver='lsqr', shrinkage='auto', tol=1e-4, ), "LDA 5"),
# #     (LDA(solver='lsqr', shrinkage=0.1, tol=1e-4, ), "LDA 6"),
# #     (LDA(solver='lsqr', shrinkage=0.5, tol=1e-4, ), "LDA 7"),
# #     (LDA(solver='lsqr', shrinkage=0.9, tol=1e-4, ), "LDA 8"),
#     (LDA(solver='eigen', shrinkage='auto', tol=1e-4, ), "LDA 9"),
# #     (LDA(solver='eigen', shrinkage=0.1, tol=1e-4, ), "LDA 10"),
# #     (LDA(solver='eigen', shrinkage=0.5, tol=1e-4, ), "LDA 11"),
# #     (LDA(solver='eigen', shrinkage=0.9, tol=1e-4, ), "LDA 12"),
# ]


#     'clf__metric': ('euclidean', 'manhattan'),
# #     'clf__shrink_threshold': (None, 0.1),
nc_clf_list = [
    (NearestCentroid(metric='euclidean', shrink_threshold=None), "NC euclidean None"),
#     (NearestCentroid(metric='euclidean', shrink_threshold=0.0),  "NC 2"),
#     (NearestCentroid(metric='euclidean', shrink_threshold=0.1),  "NC 3"),
#     (NearestCentroid(metric='euclidean', shrink_threshold=0.2),  "NC 4"),
]

In [102]:
X = sentences_df['sentence']
y = sentences_df['sentiment']

# for rs in range(101):
X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=62)
X_data = X_data.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

# for train_index, test_index in StratifiedKFold(y_data, 10):
#     X_train, X_test = X_data[train_index], X_data[test_index]
#     y_train, y_test = y_data[train_index], y_data[test_index]

X_train, X_test, y_train, y_test = X_data, X_val, y_data, y_val

clf = NearestCentroid(metric='euclidean', shrink_threshold=None)
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', clf),
])
benchmark(
    "NC metric='euclidean' shrink_threshold=None use_idf=True ngram_range=(1, 2) random_state=62",
    pipeline, X_train, X_test, y_train, y_test)[1]

________________________________________________________________________________
Training: NC metric='euclidean' shrink_threshold=None use_idf=True ngram_range=(1, 2) random_state=62
Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
train time: 0.838s
test time:  0.097s
accuracy:   0.716
matthews_corrcoef:   0.397
classification report:
             precision    recall  f1-score   support

        neg       0.61      0.64      0.63       228
        pos       0.79      0.76      0.77       391

avg / total       0.72      0.72      0.72       619

confusion matrix:
[[147  81]
 [ 95 296]]



0.39715043870970373

In [97]:
with open('sentiment/pipeline_nc.pickle', 'wb') as pipeline_nc_file:
    pickle.dump(pipeline, pipeline_nc_file)

In [98]:
with open('sentiment/nc.pickle', 'wb') as nc_file:
    pickle.dump(clf, nc_file)

In [100]:
pipeline.predict(['My puppy is cute.', 'People die in war.', 'I love my family.'])

array(['pos', 'neg', 'pos'], dtype=object)

In [103]:
X = sentences_df['sentence']
y = sentences_df['sentiment']

rs1 = []

# for rs in range(101):
X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=83)
X_data = X_data.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

for train_index, test_index in StratifiedKFold(y_data, 10):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

#     X_train, X_test, y_train, y_test = X_data, X_val, y_data, y_val

    clf = NearestCentroid(metric='euclidean', shrink_threshold=None)
    pipeline = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1, 2))),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', clf),
    ])
    rs1.append((rs, benchmark(
        "NC metric='euclidean' shrink_threshold=None use_idf=False ngram_range=(1, 2) random_state={}".format(83),
        pipeline, X_train, X_test, y_train, y_test)[1]))

________________________________________________________________________________
Training: NC metric='euclidean' shrink_threshold=None use_idf=False ngram_range=(1, 2) random_state=0
Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
train time: 0.745s
test time:  0.090s
accuracy:   0.651
matthews_corrcoef:   0.265
classification report:
             precision    recall  f1-score   support

        neg       0.51      0.58      0.55       200
        pos       0.75      0.69      0.72       358

avg / total       0.66      0.65      0.65       558

confusion matrix:
[[117  83]
 [112 246]]

________________________________________

In [104]:
max([m for n, m in rs1])

0.4155316233443187

In [105]:
max([m for n, m in rs2])

0.22252977389276088

In [107]:
dr = {}
for k in results1:
    

{0: [0.2653080923543058,
  0.2518435754189946,
  0.2994486796291157,
  0.23290195832225963,
  0.27771904345981474,
  0.27817792544716996,
  0.2179078675577204,
  0.26347712834225284,
  0.28071966346232241,
  0.34259150710125363],
 1: [0.23214172201737757,
  0.39537104314123411,
  0.26043871926187656,
  0.20750410565687474,
  0.31489862876461994,
  0.25707166024003969,
  0.34310895623782045,
  0.27549243145009034,
  0.21622799565297957,
  0.31328892207199693],
 2: [0.31121133613596103,
  0.29626773851592209,
  0.26132175715760098,
  0.16277276948371427,
  0.22048335948563477,
  0.26809116130848171,
  0.28460808245269109,
  0.26977795238482938,
  0.37431716024604672,
  0.31063971579397526],
 3: [0.29515962757031183,
  0.29136714361997568,
  0.24815281476425827,
  0.27195164422386181,
  0.30562665837965769,
  0.31367394508973756,
  0.2437130493948152,
  0.27001691883336332,
  0.23678954253385806,
  0.34148657519983089],
 4: [0.29006187914397352,
  0.35595016900636117,
  0.2967350887485774

In [76]:
results1 = {}

X = sentences_df['sentence']
y = sentences_df['sentiment']

for rs in range(101):
    result = []
    
    X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=rs)
    X_data = X_data.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_data = y_data.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)
    
    for train_index, test_index in StratifiedKFold(y_data, 10):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]

        pipeline = Pipeline([
            ('vect', CountVectorizer(ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer(use_idf=True)),
            ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None)),
        ])
        result.append(benchmark(
            "NC metric='euclidean' shrink_threshold=None use_idf=True ngram_range=(1, 2) random_state={}".format(rs),
            pipeline, X_train, X_test, y_train, y_test)[1])
    results1[rs] = result

________________________________________________________________________________
Training: NC metric='euclidean' shrink_threshold=None use_idf=True ngram_range=(1, 2) random_state=0
Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
train time: 0.742s
test time:  0.091s
accuracy:   0.651
matthews_corrcoef:   0.265
classification report:
             precision    recall  f1-score   support

        neg       0.51      0.58      0.55       200
        pos       0.75      0.69      0.72       358

avg / total       0.66      0.65      0.65       558

confusion matrix:
[[117  83]
 [112 246]]

_________________________________________

In [79]:
results1_avg = {}
for k in results1:
    results1_avg[k] = np.average(results1[k])

In [108]:
max(results1_avg.values())

0.29847500762157619

In [82]:
results1_avg

{0: 0.27100954410952094,
 1: 0.28155441844949097,
 2: 0.27594910329648575,
 3: 0.2817937919609671,
 4: 0.29245591070696575,
 5: 0.28076238756169603,
 6: 0.27161590739901603,
 7: 0.29409141625938207,
 8: 0.29113291757436127,
 9: 0.27515107461373844,
 10: 0.28338072369715778,
 11: 0.27910608801203585,
 12: 0.28401838372182092,
 13: 0.27943773379390013,
 14: 0.2867196595107861,
 15: 0.28837051929495999,
 16: 0.29816554401701051,
 17: 0.29054358215237847,
 18: 0.29326280241474534,
 19: 0.28873119276937242,
 20: 0.28938487121318857,
 21: 0.27599337683923164,
 22: 0.29544634355632188,
 23: 0.28551485051232156,
 24: 0.28202276228703904,
 25: 0.28326186572889994,
 26: 0.27764513312065159,
 27: 0.28622136318760127,
 28: 0.28948830237009704,
 29: 0.2839315679384592,
 30: 0.28581702564960704,
 31: 0.28920969613277486,
 32: 0.28090403488426408,
 33: 0.28329420811339012,
 34: 0.29434951830252032,
 35: 0.28806943547579006,
 36: 0.28204130406412209,
 37: 0.28148077210353173,
 38: 0.28772742126055117,

In [72]:
from collections import defaultdict
results1 = defaultdict(list)

X = sentences_df['sentence']
y = sentences_df['sentiment']
for rs in range(101):
    result = []
    
    X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=rs)
    X_data = X_data.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    y_data = y_data.reset_index(drop=True)
    y_val = y_val.reset_index(drop=True)
    
    for train_index, test_index in StratifiedKFold(y_data, 10):
        X_train, X_test = X_data[train_index], X_data[test_index]
        y_train, y_test = y_data[train_index], y_data[test_index]

        pipeline = Pipeline([
            ('vect', CountVectorizer(ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer(use_idf=True)),
            ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None)),
        ])
        benchmark_results = benchmark("NC metric='euclidean' shrink_threshold=None use_idf=True ngram_range=(1, 2)",
                                      pipeline, X_train, X_test, y_train, y_test)

________________________________________________________________________________
Training: NC euclidean None
Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
train time: 0.736s
test time:  0.090s
accuracy:   0.680
matthews_corrcoef:   0.314
classification report:
             precision    recall  f1-score   support

        neg       0.54      0.59      0.57       197
        pos       0.76      0.73      0.75       360

avg / total       0.69      0.68      0.68       557

confusion matrix:
[[116  81]
 [ 97 263]]

________________________________________________________________________________
Training: NC euclidean None
Pipel

In [50]:
pipeline

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])

In [48]:
clf

NearestCentroid(metric='euclidean', shrink_threshold=None)

In [51]:
with open('sentiment/pipeline_nc1.pickle', 'wb') as pipeline_nc_file:
    pickle.dump(pipeline, pipeline_nc_file)

In [52]:
with open('sentiment/nc1.pickle', 'wb') as nc_file:
    pickle.dump(clf, nc_file)

In [33]:
# ________________________________________________________________________________
# Training: LDA
# Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
#         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
#         lowercase=True, max_df=1.0, max_features=None, min_df=1,
#         ngram_range=(1, 2), preprocessor=None, stop_words=None,
#         strip..._components=None, priors=None, shrinkage=None, solver='svd',
#   store_covariance=False, tol=0.0001))])
# train time: 242.070s
# test time:  0.780s
# accuracy:   0.647
# matthews_corrcoef:   0.055
# classification report:
#              precision    recall  f1-score   support

#         neg       0.56      0.03      0.05      1097
#         pos       0.65      0.99      0.78      1995

# avg / total       0.62      0.65      0.52      3092

# confusion matrix:
# [[  28 1069]
#  [  22 1973]]

# /home/antonio/.virtualenvs/testliwc/lib/python3.4/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
#   warnings.warn("Variables are collinear.")


# ________________________________________________________________________________
# Training: LDA 5
# Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
#         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
#         lowercase=True, max_df=1.0, max_features=None, min_df=1,
#         ngram_range=(1, 2), preprocessor=None, stop_words=None,
#         strip...mponents=None, priors=None, shrinkage='auto', solver='lsqr',
#   store_covariance=False, tol=0.0001))])

# ---------------------------------------------------------------------------
# MemoryError  


# ________________________________________________________________________________
# Training: LDA 9
# Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
#         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
#         lowercase=True, max_df=1.0, max_features=None, min_df=1,
#         ngram_range=(1, 2), preprocessor=None, stop_words=None,
#         strip...ponents=None, priors=None, shrinkage='auto', solver='eigen',
#   store_covariance=False, tol=0.0001))])

# ---------------------------------------------------------------------------
# MemoryError    



# ________________________________________________________________________________
# Training: LDA 1
# Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
#         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
#         lowercase=True, max_df=1.0, max_features=None, min_df=1,
#         ngram_range=(1, 2), preprocessor=None, stop_words=None,
#         strip..._components=None, priors=None, shrinkage=None, solver='svd',
#   store_covariance=False, tol=0.0001))])
# train time: 246.366s
# test time:  0.192s
# accuracy:   0.651
# matthews_corrcoef:   0.124
# classification report:
#              precision    recall  f1-score   support

#         neg       0.54      0.14      0.22       221
#         pos       0.66      0.93      0.77       398

# avg / total       0.62      0.65      0.58       619

# confusion matrix:
# [[ 31 190]
#  [ 26 372]]

# ________________________________________________________________________________
# Training: LDA 2
# Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
#         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
#         lowercase=True, max_df=1.0, max_features=None, min_df=1,
#         ngram_range=(1, 2), preprocessor=None, stop_words=None,
#         strip...n_components=None, priors=None, shrinkage=None, solver='svd',
#   store_covariance=False, tol=0.001))])
# train time: 250.749s
# test time:  0.194s
# accuracy:   0.651
# matthews_corrcoef:   0.124
# classification report:
#              precision    recall  f1-score   support

#         neg       0.54      0.14      0.22       221
#         pos       0.66      0.93      0.77       398

# avg / total       0.62      0.65      0.58       619

# confusion matrix:
# [[ 31 190]
#  [ 26 372]]


In [ ]:
# with open('sentiment/gs.pickle', 'wb') as gs_file:
#     pickle.dump(grid_search, gs_file, protocol=2)

In [ ]:
with open('sentiment/be0.pickle', 'wb') as be_file:
    pickle.dump(be, be_file, protocol=2)

In [ ]:
results = []

for train_index, test_index in StratifiedKFold(y, K):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    benchmark_result = benchmark('NearestCentroid', be, X_train, X_test, y_train, y_test)
    results.append(benchmark_result)

In [38]:
results = []

X = sentences_df['sentence']
y = sentences_df['sentiment']
X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=100)
X_data = X_data.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
for train_index, test_index in StratifiedKFold(y_data, 10):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1, 2), max_df=1.0, stop_words='english')),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('densifier', Densifier()),
        ('clf', NearestCentroid(metric='euclidean')),
    ])
#     grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=StratifiedKFold(y_data, K))
#     grid_search.fit(X_train, y_train)
#     be = grid_search.best_estimator_
#     print('*'*40)
#     print(be.get_params())
#     print('*'*40)
    benchmark_results = benchmark('NearestCentroid', pipeline, X_train, X_test, y_train, y_test)
    
#     results.append((be, grid_search.best_score_))

________________________________________________________________________________
Training: NearestCentroid
Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        ...ier object at 0x7fe908272668>), ('clf', NearestCentroid(metric='euclidean', shrink_threshold=None))])
train time: 2.681s
test time:  0.241s
accuracy:   0.651
matthews_corrcoef:   0.227
classification report:
             precision    recall  f1-score   support

        neg       0.51      0.48      0.49       198
        pos       0.72      0.74      0.73       360

avg / total       0.65      0.65      0.65       558

confusion matrix:
[[ 95 103]
 [ 92 268]]

________________________________________________________________________________
Training: NearestCentroid
Pipeline(

In [ ]:
from collections import defaultdict
# results = defaultdict(lambda: defaultdict(list))
results = defaultdict(list)

X = sentences_df['sentence']
y = sentences_df['sentiment']
X_data, X_val, y_data, y_val = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=100)
X_data = X_data.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
for train_index, test_index in StratifiedKFold(y_data, K):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    for clf, name in clf_list:
        pipeline = Pipeline([
            ('vect', CountVectorizer(ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer()),
            ('clf', clf),
        ])
#         grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
        benchmark_results = benchmark(name, pipeline, X_train, X_test, y_train, y_test)
        results[name].append(benchmark_results)

In [ ]:
y_test.value_counts()

In [ ]:
X_data.reset_index(drop=True).head()

In [ ]:
y_data.reset_index(drop=True).head()

In [ ]:
len(X_train)

In [ ]:
X_train

In [ ]:
X_data.reindex()

In [ ]:
X_data.ix[597]

In [ ]:
X_train.ix[597]

In [ ]:
# K = 10
# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 1000, 5000, 10000, 50000),
#     'vect__ngram_range': ((1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3)),  # unigrams or bigrams or trigrams
#     'vect__stop_words': (None, stopwords.words('english')),
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2'),
# #     'clf__alpha': (1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6),
# #     'clf__penalty': ('l1', 'l2', 'elasticnet'),
# #     'clf__n_iter': (10, 50, 80),
# #     'clf__loss': ('log', 'modified_huber'),
# }
# csf_list = [
#     (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
#     (Perceptron(n_iter=50), "Perceptron"),
#     (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
#     (KNeighborsClassifier(n_neighbors=10), "kNN"),
#     (RandomForestClassifier(n_estimators=100), "Random forest")
# ]
# results = {}
# for clf, name in csf_list:
#     pipeline = Pipeline([
#         ('vect', CountVectorizer()),
#         ('tfidf', TfidfTransformer()),
#         ('clf', clf),
#     ])
#     grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
#     skf = StratifiedKFold(y, K)
#     results[name] = cross_val_score(grid_search, X, y, cv=skf, n_jobs=-1, verbose=1)

In [ ]:
# split a training set and a test set
y_train = train_labels
y_test = test_labels

print("Extracting features from the training data using a sparse vectorizer")
t0 = time()
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
X_train = vectorizer.fit_transform(data_train.sentence.tolist())
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_train.shape)
print()

print("Extracting features from the test data using the same vectorizer")
t0 = time()
X_test = vectorizer.transform(data_test.sentence.tolist())
duration = time() - t0
print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
print("n_samples: %d, n_features: %d" % X_test.shape)
print()

In [ ]:
# mapping from integer feature name to original token string
feature_names = vectorizer.get_feature_names()
opts_select_chi2 = 100

print("Extracting %d best features by a chi-squared test" %
      opts_select_chi2)
t0 = time()
ch2 = SelectKBest(chi2, k=opts_select_chi2)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)
if feature_names:
    # keep selected feature names
    feature_names = [feature_names[i] for i
                     in ch2.get_support(indices=True)]
print("done in %fs" % (time() - t0))
print()

feature_names = np.asarray(feature_names)

In [ ]:
# feature_names

In [ ]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) < 80 else s[:75] + "..."

In [ ]:
metrics.pairwise.pairwise_distances